In [ ]:
from random import random
from selenium import webdriver
from logging import raiseExceptions
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException

In [ ]:
service = Service()
options = webdriver.ChromeOptions()
options.add_extension("adblock.crx")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-setuid-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.page_load_strategy = 'eager'
prefs = {
    "profile.managed_default_content_settings.images": 2,
    "profile.managed_default_content_settings.stylesheets": 2,
    "profile.managed_default_content_settings.plugins": 2,
    "profile.managed_default_content_settings.popups": 2,
    "profile.managed_default_content_settings.geolocation": 2,
    "profile.managed_default_content_settings.notifications": 2,
    "profile.managed_default_content_settings.media_stream": 2,
    "profile.managed_default_content_settings.javascript": 1,
    "profile.managed_default_content_settings.frame": 2
}

options.add_experimental_option("prefs", prefs)
options.add_experimental_option("prefs", prefs)
options.set_capability("pageLoadStrategy", "eager")

In [ ]:
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
driver.get('https://www.justicaeleitoral.jus.br/fato-ou-boato/@@pesquisar?SearchableText=&Agencias%3Alist=AFP&periodo_inicial=&periodo_final=')
list = ['#', '\'', '/', ',', '.', '', ']', '[', '(', ')', '*', '@', '!', '?', '|', ':', ';', '&', '–', '”', '“', '…', '=', ',', '.', '–', '"']
trainNews = []
testNews = []
aux = []

for i in range(6):
    # Find all the news cards
    try:
        newsCards = WebDriverWait(driver, 30).until(
            EC.presence_of_all_elements_located(
                (By.XPATH,
                    '//div[@class="check-labels"]//img[@src="https://www.justicaeleitoral.jus.br/fato-ou-boato/++theme++fatosboatos_v2/images/logos/cards/afp.png"]'
                )
            )
        )
    except:
        raise Exception('element not found')

    for card in newsCards:
        # Scroll to the card and click
        try:
            driver.execute_script("arguments[0].scrollIntoView({behavior:'smooth',block:'center'});", card)
            WebDriverWait(driver, 120).until(EC.element_to_be_clickable(card)).click()
        except ElementClickInterceptedException:
            print("error")
            continue

        # Change the driver window
        WebDriverWait(driver, 120).until(EC.number_of_windows_to_be(2))
        driver.switch_to.window(driver.window_handles[-1])

        # Search all the news and store them
        try:
            WebDriverWait(driver, 2).until(
                EC.presence_of_element_located((By.XPATH, "//p[1]/em"))
            )

            newsElement = driver.find_element(By.XPATH, "//p[1]/em")

            aux = []
            if (random() < 0.8):
                text = newsElement.text.strip() # Add the news
                text = text.split()
                for words in text:
                    for char in list:
                        words = words.replace(char, '')
                    aux.append(words.lower())

                aux = [word for word in aux if word != '']
                aux.insert(0, 'f')
                trainNews.append(aux)
            else:
                text = newsElement.text.strip() # Add the news
                text = text.split()
                for words in text:
                    for char in list:
                        words = words.replace(char, '')
                    aux.append(words.lower())

                aux = [word for word in aux if word != '']
                aux.insert(0, 'f')
                testNews.append(aux)
        except Exception as e:
            print("notícias sem dados, pulando...")
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            continue

        # Close the window
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    # Click on the next button
    nextButton = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//a[@class="next" and contains(@href, "b_start:int=")]'))
    )
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", nextButton)
    nextButton.click()

    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//div[@class="check-labels"]'))
    )

In [ ]:
arqOut = open('./TRAINfakenewsAFP.txt', 'w', encoding='utf-8')
for news in trainNews:
    print(news, file=arqOut)
arqOut.close()

In [ ]:
arqOut = open('./TESTfakenewsAFP.txt', 'w', encoding='utf-8')
for news in testNews:
    print(news, file=arqOut)
arqOut.close()